<a href="https://colab.research.google.com/github/YaninaK/cv-segmentation/blob/b1/notebooks/03_Models_losses_%26_evalution_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corrosion detection in steel pipes

## 3. Models, Losses, Evaluation score & Training

* **The objective**:
The objective of this challenge is to train a model that have the highest possible score for the segmentation of groove defects using the provided data

In [1]:
initiate = False
if initiate:
  !git init -q
  !git clone -b b1 https://github.com/YaninaK/cv-segmentation.git -q
  !pip install -r /content/cv-segmentation/requirements_Colab.txt -q

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchsummary import summary

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Models

### UNET Model



We have chosen a compact architecture for the UNET model 🏗️ consisting of:

- 3 down-sampling blocks: (36 x 36), (18 x 18), and (9 x 9).
- 3 up-sampling blocks: (9 x 9), (18 x 18), and (36 x 36).

Here's an illustration of the UNET model:

<img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*VUS2cCaPB45wcHHFp_fQZQ.png" alt="UNET Model" width="500" height="250">



In [4]:
class UNet(nn.Module):
    def __init__(self, dropout_prob=0.5):
        super(UNet, self).__init__()

        # Encoder
        self.enc_conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.enc_conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.enc_conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.enc_conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.maxpool = nn.MaxPool2d(2, 2)

        # Additional layers in encoder
        self.enc_conv1_additional = nn.Conv2d(32, 32, 3, padding=1)
        self.enc_conv2_additional = nn.Conv2d(64, 64, 3, padding=1)

        # Dropout layers
        self.dropout = nn.Dropout2d(p=dropout_prob)

        # Decoder
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec_conv1 = nn.Conv2d(128, 64, 3, padding=1)
        self.upconv2 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.dec_conv2 = nn.Conv2d(64, 32, 3, padding=1)
        self.final_conv = nn.Conv2d(32, 1, 1)

    def forward(self, x):
        # Encoder
        x1 = torch.relu(self.enc_conv1(x))
        x1 = torch.relu(self.enc_conv1_additional(x1))
        x2 = self.maxpool(x1)
        x2 = torch.relu(self.enc_conv2(x2))
        x2 = torch.relu(self.enc_conv2_additional(x2))
        #x2 = self.dropout(x2)
        x3 = self.maxpool(x2)
        x3 = torch.relu(self.enc_conv3(x3))
        x3 = torch.relu(self.enc_conv4(x3))
        #x3 = self.dropout(x3)

        # Decoder
        x = torch.relu(self.upconv1(x3))
        x = torch.cat([x2, x], dim=1)
        x = torch.relu(self.dec_conv1(x))
        x = torch.relu(self.upconv2(x))
        x = torch.cat([x1, x], dim=1)
        x = torch.relu(self.dec_conv2(x))
        x = torch.sigmoid(self.final_conv(x))
        #(x.shape)
        return x

* Чтобы избежать переобучения модели, небольшой ```dropout``` лучше оставить, например: ```dropout_prob=0.1```. Более подходящее значение можно будет определить при подборе гиперпараметров.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Initialize the model
model = UNet()
model.to(device)
summary(model, input_size=(1, 36, 36))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 36, 36]             320
            Conv2d-2           [-1, 32, 36, 36]           9,248
         MaxPool2d-3           [-1, 32, 18, 18]               0
            Conv2d-4           [-1, 64, 18, 18]          18,496
            Conv2d-5           [-1, 64, 18, 18]          36,928
         MaxPool2d-6             [-1, 64, 9, 9]               0
            Conv2d-7            [-1, 128, 9, 9]          73,856
            Conv2d-8            [-1, 128, 9, 9]         147,584
   ConvTranspose2d-9           [-1, 64, 18, 18]          32,832
           Conv2d-10           [-1, 64, 18, 18]          73,792
  ConvTranspose2d-11           [-1, 32, 36, 36]           8,224
           Conv2d-12           [-1, 32, 36, 36]          18,464
           Conv2d-13            [-1, 1, 36, 36]              33
Total params: 419,777
Trainable params:

In [6]:
tensor = torch.rand(1,1, 36, 36).to(device)
model(tensor)

tensor([[[[0.4831, 0.4830, 0.4826,  ..., 0.4826, 0.4816, 0.4825],
          [0.4845, 0.4823, 0.4822,  ..., 0.4797, 0.4810, 0.4817],
          [0.4825, 0.4816, 0.4815,  ..., 0.4785, 0.4792, 0.4820],
          ...,
          [0.4840, 0.4817, 0.4800,  ..., 0.4820, 0.4820, 0.4815],
          [0.4831, 0.4824, 0.4812,  ..., 0.4809, 0.4813, 0.4818],
          [0.4848, 0.4819, 0.4826,  ..., 0.4837, 0.4818, 0.4823]]]],
       grad_fn=<SigmoidBackward0>)

### Attention U-Net



We have chosen to incorporate attention mechanisms into the U-Net to enhance focus on the critical regions of the input image.

Here's an illustration of the Attention U-Net architecture:

<img src="https://idiotdeveloper.com/wp-content/uploads/2021/06/attention_unet-compressed-2.jpg" alt="Attention U-Net architecture" width="500" height="250">



In [7]:
class AttentionBlock(nn.Module):
    """Attention block with learnable parameters"""

    def __init__(self, F_g, F_l, n_coefficients):
        """
        :param F_g: number of feature maps (channels) in previous layer
        :param F_l: number of feature maps in corresponding encoder layer, transferred via skip connection
        :param n_coefficients: number of learnable multi-dimensional attention coefficients
        """
        super(AttentionBlock, self).__init__()

        self.W_gate = nn.Sequential(
            nn.Conv2d(F_g, n_coefficients, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(n_coefficients)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, n_coefficients, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(n_coefficients)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(n_coefficients, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, gate, skip_connection):
        """
        :param gate: gating signal from previous layer
        :param skip_connection: activation from corresponding encoder layer
        :return: output activations
        """
        g1 = self.W_gate(gate)
        x1 = self.W_x(skip_connection)
        result = torch.add(g1, x1)
        psi = self.relu(result)
        #print(g1.shape)
        #print(x1.shape)
        psi = self.psi(psi)
        out = skip_connection * psi
        return out

In [8]:
import torch
import torch.nn as nn

class AttUNet(nn.Module):
    def __init__(self, dropout_prob=0.3):
        super(AttUNet, self).__init__()

        # Encoder
        self.enc_conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.enc_conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.enc_conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.enc_conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.maxpool = nn.MaxPool2d(2, 2)



        # Dropout layers
        self.dropout = nn.Dropout2d(p=dropout_prob)

        # Decoder
        self.upconv1 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.Att1 = AttentionBlock(F_g=32, F_l=32, n_coefficients=32)
        self.dec_conv1 = nn.Conv2d(64, 32, 3, padding=1)
        self.upconv2 = nn.ConvTranspose2d(32, 16, 2, stride=2)
        self.Att2 = AttentionBlock(F_g=16, F_l=16, n_coefficients=16)
        self.dec_conv2 = nn.Conv2d(32, 16, 3, padding=1)
        self.final_conv = nn.Conv2d(16, 1, 1)

    def forward(self, x):
        # Encoder
        e1 = torch.relu(self.enc_conv1(x)) #32x36x36
        e2 = self.maxpool(e1) #32x18x18
        e2 = torch.relu(self.enc_conv2(e2)) #64x18x18
        e2 = self.dropout(e2) #64x18x18
        e3 = self.maxpool(e2) #64x9x9
        e3 = torch.relu(self.enc_conv3(e3)) #128x9x9
        elif3 = torch.relu(self.enc_conv4(e3)) #128x9x9
        e3 = self.dropout(e3)

        # Decoder
        d2 = torch.relu(self.upconv1(e3)) #[2, 64, 18, 18]
        s2 = self.Att1(gate=d2, skip_connection=e2)
        d2 = torch.cat([s2, d2], dim=1)
        d2 = torch.relu(self.dec_conv1(d2))
        #print(d2.shape)
        d1 = torch.relu(self.upconv2(d2))
        #print(e1.shape)
        #print(d1.shape)
        s1 = self.Att2(gate=d1, skip_connection=e1)
        d1 = torch.cat([s1, d1], dim=1)
        #print(d1.shape)
        d1 = torch.relu(self.dec_conv2(d1))
        out = torch.sigmoid(self.final_conv(d1))
        #(x.shape)

        return out


### ResUNet




We have chosen to incorporate  residual connections within the architecture. These residual connections can help to alleviate the vanishing gradient problem and improve the overall performance of the network

Here's an illustration of the ResUNet architecture:

<img src="https://idiotdeveloper.com/wp-content/uploads/2022/01/MultiResUNET.png" alt="ResUNet architecture" width="500" height="250">


In [9]:
class batchnorm_relu(nn.Module):
    def __init__(self, in_c):
        super().__init__()

        self.bn = nn.BatchNorm2d(in_c)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.bn(inputs)
        x = self.relu(x)
        return x

class residual_block(nn.Module):
    def __init__(self, in_c, out_c, stride=1):
        super().__init__()

        """ Convolutional layer """
        self.b1 = batchnorm_relu(in_c)
        self.c1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, stride=stride)
        self.b2 = batchnorm_relu(out_c)
        self.c2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1, stride=1)

        """ Shortcut Connection (Identity Mapping) """
        self.s = nn.Conv2d(in_c, out_c, kernel_size=1, padding=0, stride=stride)

    def forward(self, inputs):
        x = self.b1(inputs)
        x = self.c1(x)
        x = self.b2(x)
        x = self.c2(x)
        s = self.s(inputs)

        skip = x + s
        return skip

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)
        self.r = residual_block(in_c+out_c, out_c)

    def forward(self, inputs, skip):
        x = self.upsample(inputs)
        #print(x.shape)
        x = torch.cat([x, skip], axis=1)
        x = self.r(x)
        return x

class build_resunet(nn.Module):
    def __init__(self):
        super().__init__()

        """ Encoder 1 """
        self.c11 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.br1 = batchnorm_relu(64)
        self.c12 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.c13 = nn.Conv2d(1, 64, kernel_size=1, padding=0)

        """ Encoder 2 and 3 """
        self.r2 = residual_block(64, 128, stride=2)
        #self.r3 = residual_block(128, 256, stride=2)

        """ Bridge """
        self.r4 = residual_block(128, 256, stride=2)

        """ Decoder """
        #self.d1 = decoder_block(512, 256)
        self.d2 = decoder_block(256, 128)
        self.d3 = decoder_block(128, 64)

        """ Output """
        self.output = nn.Conv2d(64, 1, kernel_size=1, padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        """ Encoder 1 """
        x = self.c11(inputs)
        x = self.br1(x)
        x = self.c12(x)
        s = self.c13(inputs)
        skip1 = x + s

        """ Encoder 2 and 3 """
        skip2 = self.r2(skip1)
        #skip3 = self.r3(skip2)

        """ Bridge """
        b = self.r4(skip2)

        """ Decoder """
        #d1 = self.d1(b, skip3)
        d2 = self.d2(b, skip2)
        d3 = self.d3(d2, skip1)

        """ output """
        output = self.output(d3)
        output = self.sigmoid(output)

        return output




* Все три модели ```UNET Model```, ```Attention U-Net``` и ```ResUNet``` могут быть задействованы в сегментации изображений, но в базовом варианте пока представлена только ```UNET Model```, поэтому остальные модели лучше показывать в отдельном ноутбуке в качестве заметок для дальнейшей работы.
* Если эксперименты проводились на всех трех моделях, модели лучше вывести в отдельные модули, которые бы импортировались в ноутбук, а в ноутбуке провести сравнительный анализ результатов экспериментов.

* В дальнейшем можно поэксперементировать с перспективными архитектурами моделей:
    * UNet++: A Nested U-Net Architecture for Medical Image Segmentation Zongwei Zhou et al., [Jul 2018](https://arxiv.org/abs/1807.10165)
    * AG-CUResNeSt: A Novel Method for Colon Polyp Segmentation. Sang et al. [Mar 2022](https://arxiv.org/abs/2105.00402)
    * Mask R-CNN. Kaiming He et al. [Jan 2018](https://arxiv.org/abs/1703.06870)
    * Vision Transformer (ViT) An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale Alexey Dosovitskiy et al.[Jun 2021](https://arxiv.org/abs/2010.11929)
    * DeiT (data-efficient image transformers)
    * VGG16-U-Net
  

## Various Losses for Training

### Dice Loss + BCE  

In [10]:
class BinaryDiceLoss(nn.Module):
    """Dice loss of binary class
    Args:
        smooth: A float number to smooth loss, and avoid NaN error, default: 1
        p: Denominator value: \sum{x^p} + \sum{y^p}, default: 2
        predict: A tensor of shape [N, *]
        target: A tensor of shape same with predict
        reduction: Reduction method to apply, return mean over batch if 'mean',
            return sum if 'sum', return a tensor of shape [N,] if 'none'
    Returns:
        Loss tensor according to arg reduction
    Raise:
        Exception if unexpected reduction
    """
    def __init__(self, smooth=1, p=2, reduction='mean'):
        super(BinaryDiceLoss, self).__init__()
        self.smooth = smooth
        self.p = p
        self.reduction = reduction
        self.bce=nn.BCELoss()

    def forward(self, predict, target):
        assert predict.shape[0] == target.shape[0], "predict & target batch size don't match"
        predict = predict.contiguous().view(predict.shape[0], -1)
        target = target.contiguous().view(target.shape[0], -1)

        num = torch.sum(torch.mul(predict, target), dim=1) + self.smooth
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.smooth
        bce_loss = self.bce(predict, target)
        loss = (1 - num / den)+bce_loss

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        elif self.reduction == 'none':
            return loss
        else:
            raise Exception('Unexpected reduction {}'.format(self.reduction))

### Focal Loss

In [11]:
class FocalLoss(nn.modules.loss._WeightedLoss):

    def __init__(self, gamma=0, size_average=None, ignore_index=-100,
                 reduce=None, balance_param=1.0):
        super(FocalLoss, self).__init__(size_average)
        self.gamma = gamma
        self.size_average = size_average
        self.ignore_index = ignore_index
        self.balance_param = balance_param
        self.bce=nn.BCELoss()

    def forward(self, input, target):
        # inputs and targets are assumed to be BatchxClasses
        assert len(input.shape) == len(target.shape)
        assert input.size(0) == target.size(0)
        assert input.size(1) == target.size(1)
        # compute the negative likelyhood
        bce_loss = self.bce(input.view(-1), target.float().view(-1))
        logpt = - bce_loss
        pt = torch.exp(logpt)
        # compute the loss
        focal_loss = -( (1-pt)**self.gamma ) * logpt
        balanced_focal_loss = self.balance_param * focal_loss
        loss=balanced_focal_loss+bce_loss
        return loss

* В базовом варианте модели функция потерь ```Binary Cross Entropy``` - ```torch.nn.BCELoss``` "из коробки".

* Если эксперименты проводились c ```Dice Loss + BCE``` и ```Focal Loss```, эти функции потерь лучше вывести в отдельные модули, которые бы импортировались в ноутбук, а в ноутбуке провести сравнительный анализ результатов экспериментов.

## Evalution Score

### Dice Score

In [12]:
def dice_coeff(prediction, target):

    mask = np.zeros_like(prediction)
    mask[prediction >= 0.5] = 1

    inter = np.sum(mask * target)
    union = np.sum(mask) + np.sum(target)
    epsilon=1e-6
    result = np.mean(2 * inter / (union + epsilon))

    return result

Формулу лучше сделать для тензоров ```pytorch```, чтобы не переходить при обучении модели с ```pytorch``` на ```numpy``` и обратно:

In [13]:
def dice(
    prediction: torch.tensor, target: torch.tensor, target_one_hot=True
) -> torch.tensor:

    if not target_one_hot:
        target = torch.eye(len(target))[target]

    mask = prediction > 0.5

    return 2 * (target * mask).sum() / (target.sum() + mask.sum() + 1e-06)

In [14]:
preds = torch.tensor(
    [
        [0.85, 0.05, 0.05, 0.05],
        [0.05, 0.85, 0.05, 0.05],
        [0.05, 0.05, 0.85, 0.05],
        [0.05, 0.05, 0.05, 0.85]
    ]
)
target = torch.tensor([0, 1, 3, 2])

dice(preds, target, False)

tensor(0.5000)

In [15]:
target_one_hot = torch.tensor([
    [1., 0., 0., 0.],
    [0., 1., 0., 0.],
    [0., 0., 0., 1.],
    [0., 0., 1., 0.]
])
dice(preds, target_one_hot)

tensor(0.5000)

Также доступна формула "из коробки" - ```torchmetrics.Dice```:

In [16]:
from torchmetrics import Dice

dice_score = Dice()
print(dice_score(preds, target))

tensor(0.5000)


Validation function

In [17]:
#Validation function
def validation(model, val_loader, criterion, device):
    model.eval()
    total_val_loss = 0
    total_val_dice_coef = 0
    sample = 0
    nb_batch = 0
    with torch.no_grad():
        with tqdm(val_loader, desc='Validation', unit='batch') as tqdm_loader:
            for images, masks,_ ,_ in tqdm_loader:
                images = images.to(device)
                masks = masks.to(device)
                masks = masks.float()
                images = images.unsqueeze(1)
                masks = masks.unsqueeze(1)
                pred = model(images)
                val_loss = criterion(pred, masks)
                y_pred = pred.data.cpu().numpy().ravel()
                y_true = masks.data.cpu().numpy().ravel()
                val_dice_coef = dice_coeff(y_pred, y_true)
                total_val_loss += val_loss.item()
                total_val_dice_coef += val_dice_coef.item()
                sample += len(images)
                nb_batch += 1
                tqdm_loader.set_postfix(loss=val_loss.item(), DiceCoef=val_dice_coef.item())
    overall_val_loss = total_val_loss / nb_batch
    overall_val_dice_coef = total_val_dice_coef / nb_batch
    print(f"Validation Loss: {overall_val_loss}")
    print(f"Validation Dice Score Coef: {overall_val_dice_coef}")
    return overall_val_loss,overall_val_dice_coef

* Из-за того что функция ```dice_coeff``` на входе и выходе не работает с ```torch.tensor```, y_pred и y_true приходится переводить в ```numpy```, переходить на ```cpu```, что замедляет скорость обучения модели. Формула ```dice``` (выше), работающая с ```torch.tensor```, позволит этого избежать.
* Если исходить из логики, что основная задача - обучение модели, а валидация - вспомогательная, вместо отдельной формулы для валидации модели, я бы предложила сделать отдельную формулу для одной эпохи обучения (```training Loop```) и использовать этот законченный блок при запуске каждой эпохи.

## Training

In [18]:
run = False
if run:
    #Model
    model = UNet()
    model.to(device)
    #Hyper Parameters
    lr = 0.001
    weight_decay = 1e-5
    betas = (0.9, 0.999)
    optimizer = Adam(model.parameters(), lr=lr)
    #criterion = FocalLoss() #FocalLoss + BCE
    #criterion =BinaryDiceLoss() #DiceLoss + BCE
    criterion = nn.BCELoss() #Binary Score Entropy
    num_epochs = 25
    train_loss=[]
    train_dice_score=[]
    validation_loss=[]
    validation_dice_score=[]
    for e in range(num_epochs):
        model.train()
        total_train_loss = 0
        total_dice_coef = 0
        sample = 0
        nb_batch = 0
        with tqdm(train_loader, desc=f'Epoch {e+1}/{num_epochs}', unit='batch') as tqdm_loader:
            for images, masks,_,_ in tqdm_loader:
                optimizer.zero_grad()
                images = images.to(device)
                masks = masks.to(device)
                masks = masks.float()
                images = images.unsqueeze(1)
                masks = masks.unsqueeze(1)
                pred = model(images)
                loss = criterion(pred, masks).mean().float()
                y_pred = pred.data.cpu().numpy().ravel()
                y_true = masks.data.cpu().numpy().ravel()
                dice_coef = dice_coeff(y_pred, y_true)
                loss.backward()
                optimizer.step()
                total_train_loss += loss.item()
                total_dice_coef += dice_coef.item()
                sample += len(images)
                nb_batch += 1
                tqdm_loader.set_postfix(loss=loss.item(), DiceCoef=dice_coef.item())

            overall_loss=total_train_loss/nb_batch
            overall_score = total_dice_coef / nb_batch
            train_loss.append(overall_loss)
            train_dice_score.append(overall_score)
        print(f"Epoch [{e+1}/{num_epochs}], Total Train Loss: {total_train_loss}")
        print(f"Epoch [{e+1}/{num_epochs}], Dice Score for Training : {overall_score}")
        # Validation
        val_loss,val_dice_coef=validation(model, val_loader, criterion, device)
        validation_loss.append(val_loss)
        validation_dice_score.append(val_dice_coef)
        # Save the model
        save_dir = "../results"
        model_path = os.path.join(save_dir, f"model_epoch_{e+1}.pt")
        torch.save(model.state_dict(), model_path)


## Рекомендации

1. Все три модели ```UNET Model```, ```Attention U-Net``` и ```ResUNet``` могут быть задействованы в сегментации изображений, но в базовом варианте задействована только ```UNET Model```, поэтому остальные модели лучше показывать в отдельном ноутбуке в качестве заметок для дальнейшей работы.
* Если эксперименты проводились на всех трех моделях, модели лучше вывести в отдельные модули, которые бы импортировались в ноутбук, а в ноутбуке провести сравнительный анализ результатов экспериментов.

2. Чтобы избежать переобучения модели ```UNET Model```, небольшой ```dropout``` лучше оставить, например: ```dropout_prob=0.1```. Более подходящее значение можно будет определить при подборе гиперпараметров.

3. В базовом варианте модели функция потерь ```Binary Cross Entropy``` (```torch.nn.BCELoss"```) -  ""из коробки"".

* Если эксперименты проводились c ```Dice Loss + BCE``` и ```Focal Loss```, эти функции потерь лучше вывести в отдельные модули, которые бы импортировались в ноутбук, а в ноутбуке провести сравнительный анализ результатов экспериментов.

4. В дальнейшем можно поэксперементировать с перспективными архитектурами моделей:
    * UNet++: A Nested U-Net Architecture for Medical Image Segmentation Zongwei Zhou et al., [Jul 2018](https://arxiv.org/abs/1807.10165)
    * AG-CUResNeSt: A Novel Method for Colon Polyp Segmentation. Sang et al. [Mar 2022](https://arxiv.org/abs/2105.00402)
    * Mask R-CNN. Kaiming He et al. [Jan 2018](https://arxiv.org/abs/1703.06870)
    * Vision Transformer (ViT) An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale Alexey Dosovitskiy et al.[Jun 2021](https://arxiv.org/abs/2010.11929)
    * DeiT (data-efficient image transformers)
    * VGG16-U-Net

5. Из-за того что функция ```dice_coeff``` на входе и выходе не работает с ```torch.tensor```, ```y_pred``` и ```y_true``` приходится переводить в ```numpy``` и переходить на ```cpu```, что замедляет скорость обучения модели. Формула ```dice``` (выше), работающая с ```torch.tensor```, позволит этого избежать. Также доступна формула "из коробки" - ```torchmetrics.Dice```.

6. Если исходить из логики, что основная задача - обучение модели, а валидация - вспомогательная, вместо отдельной формулы для валидации модели, я бы предложила сделать отдельную формулу для одной эпохи обучения (```training Loop```) и использовать этот законченный блок при запуске каждой эпохи.

7. Инициализацию модели лучше сделать в отдельной ячейке, чтобы можно было легко добавлять больше эпох к текущему запуску.

8. Для логирования метрик и значений функции потерь при обучении модели и валидации, чтобы потом выводить результаты на ```TensorBoard```, в pytorch предлагается ```torch.utils.tensorboard.SummaryWriter```.
TensorBoard: набор инструментов для визуализации TensorFlow - [здесь](https://www.tensorflow.org/tensorboard?hl=ru) ссылка.

  Код запускается при инициализации модели:
```
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/corrosion_segmentation_trainer_{}'.format(timestamp))
```
  В конце каждой эпохи логируем показатели обучения и валидации модели:
```
writer.add_scalars(
  'Training vs. Validation Loss',
  { 'Training' : avg_loss, 'Validation' : avg_vloss },
  epoch_number + 1
)
writer.flush()
```
  Пример кода для запуска ```TensorBoard``` в ```Google Colab```:
```
%load_ext tensorboard
%tensorboard --logdir='/content/cv-segmentation/notebooks/runs'
```

9. Имеет смысл отслеживать и записывать лучшие версии модели:
```
best_vloss = 1_000_000.
if avg_vloss < best_vloss:
    best_vloss = avg_vloss
    model_path = 'model_{}_{}'.format(timestamp, epoch_number)
    torch.save(model.state_dict(), model_path)
```
10. Для уменьшения ```learning rate``` когда метрики перестают улучшаться, имеет смысл реализовать ```lr_scheduler``` ```ReduceLROnPlateau``` - [здесь](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html#torch.optim.lr_scheduler.ReduceLROnPlateau) ссылка на решение "из коробки". У ```pytorch``` есть другие интересные варианты ```lr_scheduler```, их тоже можно было бы рассмотреть.

11. Чтобы обучение модели автоматически останавливалось, если после какого-то набора событий нет улучшений, в блоке обучения реализуем ```EarlyStopping``` [здесь](https://pytorch.org/ignite/generated/ignite.handlers.early_stopping.EarlyStopping.html#ignite.handlers.early_stopping.EarlyStopping) ссылка на решение из библиотеки ignite. Это будет своеобразная страховка от недостаточного обучения модели: задаем заведомо избыточное число эпох, и модель останавливается, когда на валидационнй выборке перестают улучшаться заданные метрики.

12. Повысить метрики модели помогает настройка гиперпараметров - [здесь](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html) ссылка на пример настройки гиперпараметров в документации ```pytorch```.

13. После настройки гиперпараметров полезно протестировать модель на тестовой выборке и изучить, на каких экземплярах модель ошибается. Это может дать представление, как нужно трансформировать данные или изменить архитектуру модели, чтобы улучшить метрики.

14. Формулы, задейстованные в подготовке данных, обучении модели, инфененсе должны быть покрыты тестами. Так будет удобнее поддерживать модель на протяжении ее жизненного цикла. Обычно для этих целей используют библиотеки ```pytest``` - [здесь](https://docs.pytest.org/en/stable/) ссылка, ```unittest.mock``` - [здесь](https://docs.python.org/3/library/unittest.mock.html) ссылка.

15. Еще одно направление для исследований - проанализироать насколько качественная разметка. От разметки зависит и качесвтво обучения, и метрики на тестовой выборке. Если в разметке обнаружатся какие-то системные ошибки, которые можно нивелироать, есть шанс, что метрики модели улучшатся. Другой вариант - делать поправку на качество разметки - помечать в данных способы разметки/ разметчиков, как дополнительный признак.

16. Часть рекомендаций выше реалиизована в сквозном примере в ноутбуке 04_Baseline_model.ipynb - [cсылка](https://github.com/YaninaK/cv-segmentation/blob/main/notebooks/04_Baseline_model.ipynb) на ноутбук.